# Introduction/Business Problem

## A description of the problem and a discussion of the background

Scenario: More and more Greek people visit or inhabit to Canada and more specifically in the greater area of Toronto. It will be interesting to examine if there are opportunities for entrepreneurs to invest and open a new Greek restaurant to the venues of the greater area of Toronto. It will be interesting also from an academic perspective to identify if entrepreneurs tend to spot such opportunities or they have not spotted them yet.

## A description of the data and how it will be used to solve the problem

Depending of course on the available data, the idea here is to select a place where there will be no or at least a few Greek restaurantsto ensune that there is an open spot for a Greek restaurant. In addition, I aim to identify the densely populated areas with greek people which may indicates the existance of many possible customers.

I will use the data described below:
1) (https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) information about the neighborhoods present in Toronto.

2) Geospatial_Coordinates.csv file provided to us during this course.

3) I will use the link below from wikipedia to get information about the population of Toronto (https://en.m.wikipedia.org/wiki/Demographics_of_Toronto#Ethnic_diversity).

4) Finally, I will use the foursquare platform according to the instructions of the course.